In [ ]:
!pip install qiskit

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import partial_trace, Statevector, state_fidelity
import numpy as np

In [ ]:
def create_teleportation_circuit(state_to_teleport=None):
    # Create registers
    qr = QuantumRegister(3, 'q')  # Three qubits: sender, auxiliary, receiver
    cr = ClassicalRegister(2, 'c')  # Two classical bits for measurements
    circuit = QuantumCircuit(qr, cr)
    
    # Prepare the state to teleport (if not provided, create random state)
    if state_to_teleport is None:
        circuit.rx(np.random.random() * np.pi, qr[0])
        circuit.ry(np.random.random() * np.pi, qr[0])
    else:
        circuit.initialize(state_to_teleport, qr[0])
    
    # Create entanglement between auxiliary and receiver qubits (Bell pair)
    circuit.h(qr[1])  # Hadamard gate on auxiliary qubit
    circuit.cx(qr[1], qr[2])  # CNOT with auxiliary as control, receiver as target
    
    # Perform Bell state measurement between sender and auxiliary qubits
    circuit.cx(qr[0], qr[1])
    circuit.h(qr[0])
    
    # Measure sender and auxiliary qubits
    circuit.measure(qr[0], cr[0])
    circuit.measure(qr[1], cr[1])
    
    # Apply corrections based on measurements
    with circuit.if_test((cr[1], 1)):  # If second bit is 1
        circuit.x(qr[2])
    with circuit.if_test((cr[0], 1)):  # If first bit is 1
        circuit.z(qr[2])
    
    # Save the state vector for later analysis
    circuit.save_statevector()
    
    return circuit

In [ ]:
def run_teleportation(state_to_teleport=None):
    # Create and run the circuit
    circuit = create_teleportation_circuit(state_to_teleport)
    
    # Create simulator with statevector method
    simulator = AerSimulator(method='statevector')
    
    # Run the circuit
    result = simulator.run(circuit).result()
    
    # Extract statevector after teleportation
    final_statevector = result.get_statevector()
    
    # Reduce to single-qubit state (receiver qubit)
    final_state = Statevector(final_statevector)
    receiver_state = partial_trace(final_state, [0, 1])  # Trace out sender and auxiliary qubits
    
    # If a state was provided, calculate fidelity
    if state_to_teleport is not None:
        fidelity = state_fidelity(state_to_teleport, receiver_state)
    else:
        fidelity = None  # No fidelity if random state was used
    
    print(f"Teleportation completed!")
    print(f"Receiver state: {receiver_state}")
    if fidelity is not None:
        print(f"Fidelity: {fidelity:.4f}")
    
    return circuit, receiver_state

In [ ]:
# Run the teleportation with a specific state
specific_state = [1/np.sqrt(2), 1/np.sqrt(2)]  # Equal superposition state
circuit, receiver_state = run_teleportation(specific_state)

# Print the circuit
print("\nQuantum Circuit:")
print(circuit)